# DBLP Topic Modeling & Trends Analysis

**Team Member:** Julio Amaya  
**Task:** NLP & Topic Modeling  
**Date:** December 4, 2025

## Overview

This notebook performs topic modeling on DBLP research abstracts to discover research themes and track their evolution:

1. Load and preprocess paper abstracts
2. Generate TF-IDF features
3. Train topic models (LDA and NMF)
4. Extract and label topics
5. Analyze topic evolution over time
6. Visualize topic distributions by venue and year
7. Track topic trends and novelty

---

## 1. Setup and Imports

In [ ]:
import sys
import os
import re
import unicodedata
import warnings
from pathlib import Path

# Data Science Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# NLP Libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF

# Utilities
from tqdm.notebook import tqdm
import swifter  # For faster processing

# Plotting Configuration
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 6)
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully.")

**⚠️ IMPORTANT:** If you've updated `models.py`, restart the kernel (Kernel → Restart) to reload the module with the latest changes including tqdm progress bars.

## 2. Load and Explore Data

In [ ]:
# --- CONFIGURATION ---
# Set to True to test quickly on 10% of data. 
# Set to False to run the full analysis (may take 1-2 hours on a laptop).
USE_SAMPLE_DATA = False
SAMPLE_FRACTION = 0.10
RANDOM_SEED = 42

# Modeling Config
N_TOPICS = 12
MAX_FEATURES = 10000

# --- PATH SETUP ---
# Defines paths relative to where this notebook is located
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "data"
OUTPUT_DIR = BASE_DIR / "output"
FIGURES_DIR = OUTPUT_DIR / "figures"
MODELS_DIR = OUTPUT_DIR / "models"

# Create output directories automatically
FIGURES_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

print(f"📂 Looking for data in: {DATA_DIR}")
print(f"💾 Saving results to:  {OUTPUT_DIR}")

## 3. Text Preprocessing

In [ ]:
def clean_text(text):
    """
    Standardizes text: lowercase, removes URLs/emails, 
    removes special chars, and normalizes accents.
    """
    if not isinstance(text, str) or not text:
        return ""
    
    # Lowercase and standard clean
    text = text.lower()
    text = re.sub(r'http\S+|www\.\S+|\S+@\S+', '', text)
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    # Remove accents
    text = unicodedata.normalize('NFKD', text)
    text = ''.join(ch for ch in text if not unicodedata.combining(ch))
    return text.strip()

def train_topic_model(tfidf_matrix, n_topics, algorithm='lda'):
    """Trains LDA or NMF model."""
    print(f"⚙️ Training {algorithm.upper()} model with {n_topics} topics...")
    
    if algorithm == 'lda':
        model = LatentDirichletAllocation(
            n_components=n_topics,
            random_state=RANDOM_SEED,
            learning_method='online',
            batch_size=256,
            n_jobs=-1, # Use all CPU cores
            verbose=1
        )
    else:
        model = NMF(
            n_components=n_topics,
            random_state=RANDOM_SEED,
            init='nndsvda',
            verbose=1
        )
        
    model.fit(tfidf_matrix)
    return model

def get_top_keywords(model, vectorizer, n_words=10):
    """Returns a dictionary of top words per topic."""
    feature_names = vectorizer.get_feature_names_out()
    topics_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        top_indices = topic.argsort()[-n_words:][::-1]
        topics_dict[topic_idx] = [feature_names[i] for i in top_indices]
    return topics_dict

print("✅ Functions defined.")

## 4. Generate TF-IDF Features

In [ ]:
# Check if file exists
if not (DATA_DIR / 'papers').exists() and not (DATA_DIR / 'papers.parquet').exists():
    raise FileNotFoundError(f"Could not find 'papers.parquet' in {DATA_DIR}")

print("⏳ Loading dataset...")

try:
    # Try loading as a folder (parquet partition)
    papers = pd.read_parquet(DATA_DIR / 'papers')
except:
    # Try loading as a single file
    papers = pd.read_parquet(DATA_DIR / 'papers.parquet')

print(f"✅ Total papers loaded: {len(papers):,}")

## 5. Train Topic Models

We'll train both LDA and NMF models and compare their performance.

In [ ]:
# 1. Filter for valid abstracts
df = papers[papers['abstract'].notna()].copy()
df['raw_text'] = df['title'].fillna('') + ' ' + df['abstract'].fillna('')

# 2. Sampling (Optional)
if USE_SAMPLE_DATA:
    print(f"⚠️ SAMPLING MODE: Using {SAMPLE_FRACTION*100}% of data.")
    df = df.sample(frac=SAMPLE_FRACTION, random_state=RANDOM_SEED).copy()
else:
    print("🚀 FULL MODE: Using all valid data.")

print(f"Papers to process: {len(df):,}")

# 3. Text Cleaning
print("🧹 Cleaning text (this may take a while)...")
# Swifter speeds up pandas apply using your CPU cores
df['text_clean'] = df['raw_text'].swifter.apply(clean_text)

# 4. Remove artifacts (too short)
df = df[df['text_clean'].str.len() > 30].copy()
print(f"Final count after cleaning: {len(df):,}")

In [ ]:
print("🔠 Generating TF-IDF Features...")

vectorizer = TfidfVectorizer(
    max_features=MAX_FEATURES,
    stop_words='english',
    ngram_range=(1, 2),
    min_df=5,
    max_df=0.9
)

tfidf_matrix = vectorizer.fit_transform(df['text_clean'])

print(f"TF-IDF Shape: {tfidf_matrix.shape}")

## 6. Assign Topics to Documents

Assign each paper to its dominant topic for analysis.

In [ ]:
# Train the model
# NOTE: LDA is slower but often better for coherence. NMF is faster.
lda_model = train_topic_model(tfidf_matrix, N_TOPICS, algorithm='lda')

# Extract Keywords
keywords = get_top_keywords(lda_model, vectorizer, n_words=10)

print("\n--- Discovered Topics ---")
for i, words in keywords.items():
    print(f"Topic {i}: {', '.join(words[:7])}")

## 7. Visualize Topic Distribution

In [ ]:
print("Assigning topics to documents...")

# Get dominant topic for each paper
# We process in chunks to be memory safe
chunk_size = 2000
topic_ids = []

for i in tqdm(range(0, tfidf_matrix.shape[0], chunk_size)):
    chunk = tfidf_matrix[i:i+chunk_size]
    doc_topic_dist = lda_model.transform(chunk)
    topic_ids.append(doc_topic_dist.argmax(axis=1))

df['topic_id'] = np.hstack(topic_ids)

# Map IDs to Labels
topic_labels = {i: ', '.join(words[:3]) for i, words in keywords.items()}
df['topic_label'] = df['topic_id'].map(topic_labels)

# Save processed data
output_file = OUTPUT_DIR / 'paper_topics.csv'
df[['id', 'year', 'topic_id', 'topic_label']].to_csv(output_file, index=False)

# Save Models
joblib.dump(lda_model, MODELS_DIR / 'lda_model.pkl')
joblib.dump(vectorizer, MODELS_DIR / 'tfidf_vectorizer.pkl')

print(f"✅ Data saved to: {output_file}")
print(f"✅ Models saved to: {MODELS_DIR}")

## 8. Topic Evolution Over Time

Analyze how topics have evolved and gained/lost popularity over the years.

In [ ]:
# Plot Topic Counts
plt.figure(figsize=(12, 6))
topic_counts = df['topic_id'].value_counts().sort_index()

ax = sns.barplot(x=topic_counts.index, y=topic_counts.values, palette="viridis")
ax.set_title(f"Paper Distribution per Topic (n={len(df):,})")
ax.set_xlabel("Topic ID")
ax.set_ylabel("Number of Papers")

# Save and Show
plt.savefig(FIGURES_DIR / 'topic_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Summary and Export

Save models and topic assignments for future use.

In [ ]:
if 'year' in df.columns:
    plt.figure(figsize=(14, 8))
    
    # Filter valid years (Adjust range as needed)
    temp_df = df[(df['year'] >= 2000) & (df['year'] <= 2017)]
    
    # Count per year/topic
    trends = temp_df.groupby(['year', 'topic_id']).size().unstack(fill_value=0)
    
    # Get top 5 biggest topics
    top_topics = df['topic_id'].value_counts().head(5).index
    
    # Plot
    trends[top_topics].plot(marker='o', linewidth=2, ax=plt.gca())
    
    plt.title("Evolution of Top 5 Research Topics (2000-2017)")
    plt.ylabel("Number of Papers")
    plt.legend([topic_labels[i] for i in top_topics], title="Top Topics", bbox_to_anchor=(1.05, 1))
    plt.tight_layout()
    
    # Save
    plt.savefig(FIGURES_DIR / 'topic_evolution.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("Year column not found, skipping trend plot.")